In [ ]:
import dspy
import os
from dotenv import load_dotenv
from pydantic import BaseModel
from typing import List
from datetime import datetime


load_dotenv()

GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

lm = dspy.LM('gemini/gemini-2.0-flash-exp', api_key=GEMINI_API_KEY)
dspy.configure(lm=lm)

# lm("Say this is a test!", temperature=0.7)  # => ['This is a test!']
# lm(messages=[{"role": "user", "content": "Say this is a test!"}])  # => ['This is a test!']

math = dspy.ChainOfThought("question -> answer: float")
math(question="Two dice are tossed. What is the probability that the sum equals two?")

/home/devastatingrpg/miniconda3/envs/dev/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Prediction(
    reasoning='When two dice are tossed, there are 6 possible outcomes for each die, resulting in a total of 6 * 6 = 36 possible outcomes. The only way to get a sum of 2 is if both dice show a 1. This is only one specific outcome out of the 36 possible outcomes. Therefore, the probability is 1/36.',
    answer=0.027777777777777776
)

In [ ]:
class Entity(BaseModel):
    entity: str
    type: str

class EventEntity(BaseModel):
    entity: str
    type: str
    role: str

class Event(BaseModel):
    type: str
    date: str
    location: str
    entities: List[EventEntity]

class ExtractEntities(dspy.Signature):
    """Extract structEntitiesured from text."""

    text: str = dspy.InputField()

    entities: List[Entity] = dspy.OutputField(desc="a list of entities and their metadata")

class ExtractEvents(dspy.Signature):
    """Extract Event type, date, location and participating entities (along with their role in the specific event) information from text, current date and given entities."""

    text: str = dspy.InputField()
    entities: List[Entity] = dspy.InputField(desc="a list of entities and their metadata")
    current_date: str = dspy.InputField(desc="the current date to convert relative dates like 'today', 'yesterday', 'tomorrow' to actual dates")
    
    events: List[Event] = dspy.OutputField(desc="a list of events and their metadata with fields: type, date (convert relative dates like 'today', 'yesterday', 'tomorrow' to actual dates), location, entities (fetched from input)")


class KnowledgeExtraction(dspy.Module):
    def __init__(self):
      self.cot1 = dspy.ChainOfThought(ExtractEntities)
      self.cot2 = dspy.ChainOfThought(ExtractEvents)

    def forward(self, text):
        entities = self.cot1(text=text)
        current_date = datetime.now().strftime('%Y-%m-%d')
        events = self.cot2(text=text, entities=entities, current_date=current_date)
        return events
    
module = KnowledgeExtraction()

text = "Apple Inc. announced its latest iPhone 14 today." \
    "The CEO, Tim Cook, highlighted its new features in a press release."
response = module(text=text)
print(response)

Prediction(
    reasoning='The text describes an announcement event by Apple Inc. regarding their new product, iPhone 14. The date of the announcement is "today", which translates to 2025-01-18 based on the current date. The location is not explicitly mentioned, but we can assume it\'s a general announcement, so we\'ll use "global". The entities involved are Apple Inc. (company), iPhone 14 (product), and Tim Cook (person). Tim Cook\'s role is that of a speaker or presenter.',
    events=[Event(type='announcement', date='2025-01-18', location='global', entities=[EventEntity(entity='Apple Inc.', type='company', role='organization'), EventEntity(entity='iPhone 14', type='product', role='product'), EventEntity(entity='Tim Cook', type='person', role='speaker')])]
)


In [21]:
lm.inspect_history()





[2025-01-18T11:29:46.307701]

System message:

Your input fields are:
1. `text` (str)
2. `entities` (list[Entity]): a list of entities and their metadata

Your output fields are:
1. `reasoning` (str)
2. `events` (list[Event]): a list of events and their metadata with fields: type, date (convert relative dates like 'today', 'yesterday', 'tomorrow' to actual dates), location, entities (fetched from input)

All interactions will be structured in the following way, with the appropriate values filled in.

Inputs will have the following structure:

[[ ## text ## ]]
{text}

[[ ## entities ## ]]
{entities}

Outputs will be a JSON object with the following fields.

{
  "reasoning": "{reasoning}",
  "events": "{events}        # note: the value you produce must be pareseable according to the following JSON schema: {\"$defs\": {\"Event\": {\"properties\": {\"type\": {\"title\": \"Type\", \"type\": \"string\"}, \"date\": {\"title\": \"Date\", \"type\": \"string\"}, \"location\": {\"title\": \"L